In [130]:
# import sys
# !{sys.executable} -m pip install imbalanced-learn --user
# !{sys.executable} -m pip install hyperopt --user

In [131]:
from time import time
from sklearn.model_selection import GridSearchCV
# from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from copy import deepcopy

from newsgac import config
from newsgac.genres import genre_codes
from newsgac.learners import learners, LearnerSVC, LearnerNB, LearnerXGB, LearnerGB, LearnerMLP, LearnerRF, LearnerLGBM
from newsgac.pipelines.get_sk_pipeline import get_sk_pipeline
from newsgac.pipelines.utils import report
from sklearn.neural_network import MLPClassifier

from newsgac import database
from newsgac.data_sources import DataSource
from newsgac.pipelines import Pipeline

import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/newsgac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [132]:
[d.display_title for d in DataSource.objects.all()]

[u'N2BGS Training',
 u'N2BGS Testing',
 u'Linked NRC (2930)',
 u'BOW + 5 features (N2BGS train)',
 u'BOW + 5 features (N2BGS test)',
 u'nrc-1950-1994-a.txt',
 u'nrc-1950-1994-b.txt',
 u'nrc-1950-1994-c.txt',
 u'nrc-1950-1994-d.txt',
 u'nrc-1950-1994-e.txt',
 u'telegraaf-1950-1994-a.txt',
 u'telegraaf-1950-1994-b.txt',
 u'telegraaf-1950-1994-c.txt',
 u'telegraaf-1950-1994-d.txt',
 u'telegraaf-1950-1994-e.txt',
 u'volkskrant-1950-1995-a.txt',
 u'volkskrant-1950-1995-b.txt',
 u'volkskrant-1950-1995-c.txt',
 u'volkskrant-1950-1995-d.txt',
 u'volkskrant-1950-1995-e.txt',
 u'nrc-1965.txt',
 u'nrc-1985.txt',
 u'BOW + 9 features (N2BGS train)',
 u'BOW + 9 features (N2BGS test)',
 u'Linked NRC (2930/9 features)',
 u'BOW Train unique (9 features)',
 u'BOW Test unique (9 features)',
 u'Linked NRC (unique/9 features)',
 u'BOW Train unique (N3BGS/9 features)',
 u'N3BGS FROG Test',
 u'N3BGS FROG Train',
 u'BOW Test unique (N3BGS/9 features)',
 u'BOW Test unique (N3BGS/9 features/collapsed)',
 u'Link

In [212]:
# n2bgs balanced frog experiments: 2:rf 3:svc 4:xgb 5:nb 6:mlp
# n2bgs balanced bow experiments: 7:rf 8:svc 9:xgb 10:nb 11:mlp
# n3bgs balanced frog: 32:rf 33:svc 34:xgb 43:nb 44:mlp 
# n3bgs balanced bow: 45:rf 46:svc 47:xgb 48 49 
# unbalanced svc: 40
# combined svc: 42
p = Pipeline.objects.all()[44]
print 'DataSource: ' + p.data_source.display_title
print 'NLP Tool: ' + p.nlp_tool.name
print 'Classifier: ' + p.learner.name
print 'Task status: ' + str(p.task.status)

DataSource: N3BGS FROG Train
NLP Tool: Frog
Classifier: Multi-Layer Perceptron
Task status: Status.SUCCESS


In [213]:
len(p.data_source.articles)

857

In [215]:
NBROFINTERESTINGGENRES = 8

results = p.result.confusion_matrix.get()
nbrOfArticles = len(p.data_source.articles)
totalScore = 0
maxDeviation = 0.0
totalDeviation = 0.0
for genre in range(0,NBROFINTERESTINGGENRES):
    correct = results[genre][genre]
    gold = sum(results[genre])
    guessed = sum([x[genre] for x in results])
    fp = abs(guessed-correct)
    fn = abs(gold-correct)
    score = abs(fp-fn)
    totalScore += score
    deviation = float(abs(guessed-gold))/float(gold)
    maxDeviation = max(maxDeviation,deviation)
    totalDeviation += deviation
print(float(totalScore)/float(nbrOfArticles),totalDeviation/NBROFINTERESTINGGENRES,maxDeviation)

(0.03733955659276546, 0.07424877707896575, 0.18518518518518517)


In [121]:
skp = p.get_sk_pipeline()
skp_opt = deepcopy(skp)
skp_opt.steps.pop()
skp_opt.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7fb69b2afcd0>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('Basic', <newsgac.nlp_tools.transformers.ExtractBasicFeatures object at 0x7fb69b2af890>), ('Quote', <newsgac.nlp_tools.transformers.ExtractQuotes object at 0x7fb69b2aff10>), ('Sentiment', Pipeline(memory=None,
       steps=[('RemoveQuotes', <newsgac.nlp_tools.transformers.RemoveQuot...10990>), ('Frog', <newsgac.nlp_tools.models.frog.FrogFeatureExtractor object at 0x7fb696810f90>)]))],
         transformer_weights=None)),
 ('RobustScaler',
  RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=False,
         with_scaling=True))]

In [122]:
skp_opt.steps.pop()
skp_opt.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7fb69b2afcd0>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('Basic', <newsgac.nlp_tools.transformers.ExtractBasicFeatures object at 0x7fb69b2af890>), ('Quote', <newsgac.nlp_tools.transformers.ExtractQuotes object at 0x7fb69b2aff10>), ('Sentiment', Pipeline(memory=None,
       steps=[('RemoveQuotes', <newsgac.nlp_tools.transformers.RemoveQuot...10990>), ('Frog', <newsgac.nlp_tools.models.frog.FrogFeatureExtractor object at 0x7fb696810f90>)]))],
         transformer_weights=None))]

In [123]:
texts = np.array([article.raw_text for article in p.data_source.articles])
labels = np.array([article.label for article in p.data_source.articles])

X = skp_opt.fit_transform(texts)

In [124]:
# get the test dataset for optimization accuracy
texts_test = np.array([article.raw_text for article in DataSource.objects.all()[1].articles])
labels_test = np.array([article.label for article in DataSource.objects.all()[1].articles])

X_test = skp_opt.transform(texts_test)

In [125]:
X_train = X
y_train = labels
y_test = labels_test

In [126]:
y_test.shape

(91,)

In [127]:
# paste these values in corresponding variables
# {'solver': 'lbfgs', 'activation': 'relu', 'max_iter': 312, 'batch_size': 26, 'alpha': 0.3951743267863599, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (300, 100)}
mlp_opt = MLPClassifier(
            hidden_layer_sizes = (300, 100),
            activation = 'relu',
            solver = 'lbfgs',
            alpha = 0.3951743267863599,
            learning_rate = 'adaptive',
            batch_size = 26,
            max_iter = 312,
            shuffle = True,
            random_state = 42,
            warm_start = False,
            early_stopping = True,
            validation_fraction = 0.1,
    )

fitted_model = mlp_opt.fit(X=X_train,y=y_train)

In [128]:
y_score = fitted_model.predict_proba(X_test)
y_score

array([[7.57775551e-02, 4.58807271e-02, 3.76642038e-02, ...,
        1.20391703e-02, 6.88251059e-03, 1.10685582e-01],
       [1.67570580e-01, 2.45996899e-02, 7.73804966e-03, ...,
        1.11621285e-03, 6.84309013e-02, 1.57817689e-02],
       [8.59694800e-03, 2.52998104e-02, 4.59234410e-02, ...,
        4.90843109e-02, 9.45176125e-05, 2.44475717e-01],
       ...,
       [1.16618429e-01, 3.94325453e-02, 2.52911836e-02, ...,
        5.33519373e-03, 2.29729863e-02, 6.14336690e-02],
       [7.40719464e-06, 2.41628047e-01, 1.69362424e-01, ...,
        9.41950222e-02, 2.84384265e-03, 9.97169922e-02],
       [2.96781634e-01, 2.63387419e-02, 5.20116969e-04, ...,
        2.17563074e-04, 1.33761866e-01, 2.58157427e-02]])

In [129]:
test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
test_acc

0.26373626373626374